In [37]:
from tkinter import *
from tkinter import ttk
import time
import sys
import requests
import json
import math
import sqlite3
conn = sqlite3.connect('C:/app/bd_apt.db')
cursor1 = conn.cursor()
cursor3 = conn.cursor()

In [38]:
def get_all(myjson, key):
    if type(myjson) == str:
        myjson = json.loads(myjson)
    if type(myjson) is dict:
        for jsonkey in myjson:
            if type(myjson[jsonkey]) in (list, dict):
                get_all(myjson[jsonkey], key)
            elif jsonkey == key:
                return(myjson[jsonkey])

In [39]:
def find222(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try: results.append(a_dict[id])
        except KeyError: pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict)  # Return value ignored.
    return results

In [40]:
def distance(a,b,c,d):
    #pi - число pi, rad - радиус сферы (Земли)
    rad = 6372795

    #координаты двух точек
    llat1 = float(a)
    llong1 = float(b)

    llat2 = float(c)
    llong2 = float(d)

    #в радианах
    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.

    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2 * sdelta, 2)+math.pow((cl1 * sl2) - (sl1 * cl2 * cdelta), 2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad

    return dist
    print('Distance >> %.0f' % dist, ' [meters]')

In [41]:
# Получаем поля из формы
def click_button():
    A = e1.get()
    B = e2.get()
    C = e3.get()
    out(A,B,C)

In [53]:
# Делаем запросы к БД, вызываем др функции, выводим ответ
def out(street, house, lec):
    naz = lec
    # Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
    cursor3.execute("""SELECT aktiv_id FROM lec WHERE name=(?)""", (naz,))
    results3 = cursor3.fetchall()
    a = results3[0]
    b = str(a[0])
    cursor1.execute("SELECT lec.name,lec.volume,MIN(price),contry.name,naz_a.name,naz_a.id FROM contry,lec,naz_a WHERE contry_id = contry.id AND naz_a.id=naz_a_id AND lec.aktiv_id= (?)",
                    (b,))
    results1 = cursor1.fetchall()
    
    adr, metrs = dist(street, house, results1[0][5])
    otvet = "Лекарство: " + results1[0][0] + "\nСтрана производства: " + results1[0][3]
    otvet += "\nАптека: " + results1[0][4] + " \nЦена: " + str(results1[0][2])
    otvet += "рублей\nАдресс: " + adr + "\nРасстояние от указанного адресса: " + str(metrs)
    print(otvet)
    t1.insert(END, otvet)
    pass

In [70]:
# Расчет расстояния
def dist(street, house, apt_id):
    n = ''
    me = 99999
    adr = 'https://geocode-maps.yandex.ru/1.x/?format=json&geocode=Россия, Свердловская область, Екатеринбург,'
    adr1 = adr + street + ', ' + str(house)
    page1 = requests.get(adr1)
    mass1 = find222('lowerCorner', page1.text)
    koordinata1 = mass1[0].split(" ")
    
    cursor1.execute("""SELECT name,adress FROM apt WHERE naz_a_id=(?)""", (apt_id, ))
    results1 = cursor1.fetchall()
    for r in results1:
        mass2 = r[1]
        koordinata2 = mass2.split(" ")
        metrs = distance(koordinata1[0], koordinata1[1], koordinata2[0], koordinata2[1])
        if me> metrs:
            me = metrs
            n = r[0]
            
    
    return n,me

In [71]:
window = Tk()
window.geometry('650x550')

tf = Frame(window)
tf.pack()

bf = Frame(window)
bf.pack(side=BOTTOM)

l1 = Label(tf, text='Ваш адресс:')
l1.config(font=('Verdana',25))
l1.pack(side=LEFT)

l2 = Label(tf, text='Город')
l2.config(font=('Verdana',25))
l2.pack()

c1 = ttk.Combobox(tf, values='Екатеринбург')
c1.pack()
c1.current(0) 

l3 = Label(tf, text='Улица')
l3.config(font=('Verdana',25))
l3.pack()
e1 = Entry(tf, width = 10,text='Ленина')
e1.pack()
e1.insert(0, "Ленина")

l4 = Label(tf, text='Дом')
l4.config(font=('Verdana', 25))
l4.pack()
e2 = Entry(tf, width=10)
e2.pack()
e2.insert(0, "10")

l5 = Label(window, text='Необходимый препарат')
l5.config(font=('Verdana', 25))
l5.pack()
e3 = Entry(window, width=10)
e3.pack()
e3.insert(0, "АЗИТРОМИЦИН")

t1 = Text(bf, height=10, width=80)
t1.pack()


b1 = Button(bf, text='Найти', command=click_button)
b1.config(font=('Verdana', 25), width=10, height=2)
b1.pack()

window.mainloop()

['60.583254', '56.834057']
60.604059 56.779614
60.600574 56.849772
60.608075 56.802751
Лекарство: АЗИТРОМИЦИН
Страна производства: Россия
Аптека: apteka.ru 
Цена: 30рублей
Адресс: Свердлова, 11
Расстояние от указанного адресса: 2108.9815831765554


In [72]:
adr = 'https://geocode-maps.yandex.ru/1.x/?format=json&geocode=Россия, Свердловская область, Екатеринбург,'
cursor1.execute("""SELECT name,adress FROM apt WHERE naz_a_id=(?)""", (1, ))
results1 = cursor1.fetchall()
for r in results1:
    adr2 = adr + r[1]
    print(adr2)

https://geocode-maps.yandex.ru/1.x/?format=json&geocode=Россия, Свердловская область, Екатеринбург,60.604059 56.779614
https://geocode-maps.yandex.ru/1.x/?format=json&geocode=Россия, Свердловская область, Екатеринбург,60.600574 56.849772
https://geocode-maps.yandex.ru/1.x/?format=json&geocode=Россия, Свердловская область, Екатеринбург,60.608075 56.802751
